In [1]:
from numpy import load, linspace
from SpiPy.Models.SAR import *
from DTO.Database import *

2023-05-25 22:05:06,556 - numexpr.utils - INFO - NumExpr defaulting to 8 threads.


In [2]:
db_manager = HLDatabase(bWorkLaptop=False)
pollution = db_manager.get_table(table_name='Train-Pollution')
pollution.set_index('Date', inplace=True, drop=True)
mSpatial = db_manager.get_table(table_name='Train-Weight Matrix')
mSpatial = mSpatial.drop(labels='index', axis=1)

tWind = load(r"../DTO/train_tWind.npy")
tZ = load(r"../DTO/train_tZ.npy")

In [3]:
N = len(pollution.columns)
initial_params = [0.5]*N + [0.2] + [0.5] + list(pollution.var().values) + [0.5]
bounds = [(-1, 1)]*N + [(-1, 1)] + [(0, 1)] + [(1, 1000)]*N + [(0, 1)]
zeta_values = linspace(0, 1, 11)

In [4]:
tZ_1 = tZ[8000:9000, :, :]
pollution_1 = pollution.values[8000:9000, :]
tWind_1 = tWind[8000:9000, :, :]

In [ ]:
optimizer = QMLEOptimizer(
    initial_params=initial_params,
    bounds=bounds,
    weight_matrix=mSpatial,
    wind_tensor=tWind_1,
    exog=pollution_1,
    Z=tZ_1,
    method='DE'
)

optimizer.fit()
best_params = optimizer.get_best_params()

print("Best Parameters:")
print("alpha:", best_params[N])
print("rho:", best_params[N+1])
print("zeta:", best_params[-1])
print("phi:", best_params[:N])
print("Sigma:", best_params[N+2:-1])